# OpenAI Agents SDKを使用したマルチエージェント・オーケストレーション：金融ポートフォリオ分析の例

## はじめに

*このガイドは、OpenAIモデルとLLMエージェントに既に精通しており、実世界の複雑なタスクに対してエージェントチームをオーケストレーションする方法を知りたい読者を対象としています。*

**学習内容**

このノートブックでは、OpenAI Agents SDKを使用して複雑なマルチエージェント協調システムを設計・実装する方法を学びます。具体的には、以下の方法を学びます：
- 複数の専門エージェント（マクロ、ファンダメンタル、クオンツ）がポートフォリオマネージャーエージェントの下で協力し、困難な投資調査問題を解決するワークフローの構築
- 中央エージェントが特定のサブタスクのためのツールとして他のエージェントをオーケストレーションし、呼び出す「エージェントをツールとして使用する」アプローチの使用
- 単一の統合されたワークフローでSDKがサポートするすべての主要なツールタイプ（カスタムPython関数、Code InterpreterやWebSearchなどの管理ツール、外部MCPサーバー）の活用
- エージェント的パターンにおけるモジュール性、並列性、観測可能性のベストプラクティスの適用

**なぜこれが重要なのか**

「エージェントをツールとして使用する」パターンは、透明性があり、監査可能で、スケーラブルなマルチエージェント協調を構築するための強力な方法です。この例では、OpenAI Agents SDKを使用して深い専門性、並列実行、堅牢なオーケストレーションを組み合わせる方法を実証します。

このガイドの終わりまでに、調査、分析、または専門家の協力から恩恵を受ける任意の複雑なタスクのための独自のマルチエージェントワークフローを構築するための明確な設計図を手に入れることができます。

---

## 目次

1. [マルチエージェント協調とは？](#what-is-multi-agent-collaboration)
2. [協調パターン：ハンドオフ vs エージェント・アズ・ツール](#collaboration-patterns-handoff-vs-agent-as-tool)
3. [アーキテクチャ概要](#architecture-overview)
4. [サポートされるツールタイプ](#supported-tool-types)
5. [セットアップ](#setup)
6. [ワークフローの実行](#running-the-workflow)
7. [ヘッドポートフォリオマネージャー（PM）エージェント](#the-head-portfolio-manager-pm-agent)
8. [ヘッドポートフォリオマネージャーエージェントの詳細](#breaking-down-the-head-portfolio-manager-agent)
9. [出力例](#example-output)
10. [エージェント構築時のベストプラクティス](#best-practices-when-building-agents)
11. [参考資料とベストプラクティス](#further-reading--best-practices)

---

## マルチエージェント協調とは？

**マルチエージェント協調**とは、複数の自律的なエージェント（LLMの「ノード」）が連携して、単一のエージェントでは処理が困難な包括的な目標を達成することを意味します。一つの巨大なプロンプトではなく、各エージェントが特定のサブタスクや専門分野を担当し、オーケストレーション層がこれらのエージェント「ノード」を一貫性のあるワークフローに接続します。このアプローチは複雑なシステムに有用です。例えば、金融分析では、マクロ経済分析、企業のファンダメンタル分析、定量的シグナル分析に分割し、それぞれを異なる専門エージェントが処理することができます。エージェント間で情報を共有し、それらの結果を組み合わせて最終的な成果を生み出します。

### コラボレーションパターン：ハンドオフ vs エージェント・アズ・ツール

OpenAI Agents SDKは、エージェントが連携して動作する複数のパターンをサポートしています：

- **ハンドオフコラボレーション：** あるエージェントが問題の途中で他のエージェントに制御を_ハンドオフ_することができます。ハンドオフアーキテクチャでは、各エージェントは他のエージェントについて知っており、より適切なエージェントに処理を委ねるタイミングを決定できます。これはオープンエンドな会話型ワークフローには柔軟性がありますが、タスクの全体像を把握することが困難になる場合があります。[SDKドキュメントで詳細を読む。](https://openai.github.io/openai-agents-python/handoffs/)

- **エージェント・アズ・ツール：** このアプローチでは、1つのエージェント（多くの場合、中央プランナーまたはマネージャー）が**他のエージェントをツールのように呼び出します**。サブエージェントは会話を引き継ぐことはありません。代わりに、メインエージェントが特定のサブタスクのためにそれらを呼び出し、その結果を組み込みます。このモデルは単一の制御スレッドを維持し（メインエージェントがすべてを統制）、調整を簡素化する傾向があります。**このリポジトリはエージェント・アズ・ツールモデルを使用しています：** Portfolio Managerエージェントが主導権を握り、専門知識が必要な時に他の専門エージェントをツールとして使用します。この選択により、全体的な推論が透明に保たれ、サブタスクの並列実行が可能になり、複雑な分析に理想的です。

これらのコラボレーションパターンの詳細については、[OpenAI Agents SDKドキュメント](https://openai.github.io/openai-agents-python/multi_agent/)を参照してください。

---

## アーキテクチャ概要

私たちのシステムは**ハブ・アンド・スポーク設計**に従っています。**Portfolio Managerエージェント**がハブ（中央コーディネーター）であり、**専門エージェント**がスポークです。ユーザーのクエリ（例：「計画されている金利引き下げは私のGOOGL保有株にどのような影響を与えるでしょうか？」）は最初にPortfolio Managerに送られます。Portfolio Managerエージェントは問題を分解し、適切な専門エージェントに委任するようプロンプトされています。各専門エージェントを呼び出し可能なツールとして扱い、分析の担当部分について呼び出します。3つすべてがPortfolio Managerに報告し、Portfolio Managerがユーザーに対する最終的な回答を統合します。

![Multi-Agent Investment Report Workflow](../../../images/multi_agent_collab_agent_architecture.png)

---

## サポートされているツールタイプ

Agents SDKの主要な利点の一つは、エージェントが使用できる**ツール**を定義する際の柔軟性です。ツールは、シンプルなPython関数から外部サービスまで幅広く対応できます。このプロジェクトでは以下を使用しています：

- **MCP（Model Context Protocol）サーバー：** エージェントを外部ツールやデータソースに標準化された方法で接続するために使用されます。このプロジェクトでは、Yahoo Financeデータ用のローカルMCPサーバーを使用しています（`mcp/yahoo_finance_server.py`を参照）。[詳細はこちら：OpenAI MCP docs](https://openai.github.io/openai-agents-python/mcp/) | [MCP仕様](https://modelcontextprotocol.io/)

- **OpenAI管理ツール：** 管理ツールは、OpenAIが提供する組み込みのホスト型ツールで、カスタム実装を必要としません。**Code Interpreter**（定量的/統計的分析用）や**WebSearch**（最新のニュースやデータ用）など、すぐに使える強力な機能を提供します。これらのツールは統合が簡単で、OpenAIによってメンテナンスされており、追加のセットアップなしにコード実行やリアルタイム情報取得などの高度なアクションをエージェントが実行できるようにします。

- **カスタムツール：** カスタムツールは、あなたが定義してエージェント用のツールとして登録するPython関数です。Agents SDKはこれを簡単にします：関数にデコレータを付けるだけで、SDKが自動的にその名前、docstring、入力スキーマを抽出します。これは、ドメイン固有のロジック、データアクセス、ワークフロー拡張に最適です。
  私たちのプロジェクトでは、FRED経済データにアクセスする（[FRED APIを参照](https://fred.stlouisfed.org/docs/api/api_key.html)）ためと、ファイルシステム操作を実行するためにカスタムツールを使用しています。

カスタムツールは、組み込みツールや管理ツールを超えてエージェントの機能を拡張する完全な柔軟性を提供します。[関数ツールに関するSDKドキュメントを参照してください。](https://openai.github.io/openai-agents-python/tools/#function-tools)

> **さらにツールを追加したいですか？** SDKは、ウェブ検索、ファイル検索、コード実行など、幅広いツールタイプをサポートしています。[SDKドキュメントでサポートされているツールの完全なリストを参照してください。](https://openai.github.io/openai-agents-python/tools/)

---

## セットアップ

In [ ]:
# Install required dependencies
!pip install -r requirements.txt

**ワークフローを実行する前に、環境変数を設定してください：**
- `OPENAI_API_KEY`（OpenAIアクセス用）
- `FRED_API_KEY`（FRED経済データ用、[FRED APIキーの手順](https://fred.stlouisfed.org/docs/api/api_key.html)を参照）

In [ ]:
import os

missing = []
if not os.environ.get('OPENAI_API_KEY'):
    missing.append('OPENAI_API_KEY')
if not os.environ.get('FRED_API_KEY'):
    missing.append('FRED_API_KEY')

if missing:
    print(f"Missing environment variable(s): {', '.join(missing)}. Please set them before running the workflow.")
else:
    print("All required API keys are set.")

---

## ワークフローの実行

質問は好きなように編集してください。ただし、精度を向上させるために日付フィールドは保持してください！

<div style="border-left: 4px solidrgb(0, 0, 0); padding: 0.5em; background:rgb(255, 229, 229);">
<strong>免責事項:</strong> この例は教育目的のみです。投資判断を行う前に、資格を持った金融専門家にご相談ください。
</div>

ワークフローは、ユーザーリクエストをHead Portfolio Manager（PM）エージェントに送信することで開始されます。PMエージェントはプロセス全体を統括し、必要に応じて専門エージェントやツールに委任します。OpenAI Tracesを使用してワークフローをリアルタイムで監視できます。これにより、すべてのエージェントとツール呼び出しの詳細な可視性が提供されます。

以下のコードの`question`を任意の内容に編集してください。ただし、精度を向上させるためにdateフィールドは保持してください！

<div style="border-left: 4px solid #f39c12; padding: 0.5em; background: #fffbe6;">
<strong>注意:</strong> タスクの複雑さによっては、このリクエストは最大10分かかる場合があります。
</div>

In [ ]:
import datetime
import json
import os
from pathlib import Path
from contextlib import AsyncExitStack
from agents import Runner, add_trace_processor, trace
from agents.tracing.processors import BatchTraceProcessor
from utils import FileSpanExporter, output_file
from investment_agents.config import build_investment_agents
import asyncio

add_trace_processor(BatchTraceProcessor(FileSpanExporter()))

async def run_workflow():
    if "OPENAI_API_KEY" not in os.environ:
        raise EnvironmentError("OPENAI_API_KEY not set — set it as an environment variable before running.")

    today_str = datetime.date.today().strftime("%B %d, %Y")
    question = (
        f"Today is {today_str}. "
        "How would the planned interest rate reduction effect my holdings in GOOGL if they were to happen?"
        "Considering all the factors effecting its price right now (Macro, Technical, Fundamental, etc.), what is a realistic price target by the end of the year?"
    )
    bundle = build_investment_agents()

    async with AsyncExitStack() as stack:
        for agent in [getattr(bundle, "fundamental", None), getattr(bundle, "quant", None)]:
            if agent is None:
                continue
            for server in getattr(agent, "mcp_servers", []):
                await server.connect()
                await stack.enter_async_context(server)

        print("Running multi-agent workflow with tracing enabled...\n")
        with trace(
            "Investment Research Workflow",
            metadata={"question": question[:512]}
        ) as workflow_trace:
            print(
                f"\n🔗 View the trace in the OpenAI console: "
                f"https://platform.openai.com/traces/trace?trace_id={workflow_trace.trace_id}\n"
            )

            response = None
            try:
                response = await asyncio.wait_for(
                    Runner.run(bundle.head_pm, question, max_turns=40),
                    timeout=1200
                )
            except asyncio.TimeoutError:
                print("\n❌ Workflow timed out after 20 minutes.")

            report_path = None
            try:
                if hasattr(response, 'final_output'):
                    output = response.final_output
                    if isinstance(output, str):
                        data = json.loads(output)
                        if isinstance(data, dict) and 'file' in data:
                            report_path = output_file(data['file'])
            except Exception as e:
                print(f"Could not parse investment report path: {e}")

            print(f"Workflow Completed Response from Agent: {response.final_output if hasattr(response, 'final_output') else response}, investment report created: {report_path if report_path else '[unknown]'}")

# In a Jupyter notebook cell, run:
await run_workflow()

---

## Head Portfolio Manager Agentの詳細解説

Head Portfolio Manager（PM）agentは、ワークフロー全体のオーケストレーターです。それぞれ異なる専門分野に焦点を当てた4つのスペシャリストagentのセットを調整します。この設計は意図的なものです：単一のagentにあらゆる責任を過負荷させると、浅く汎用的な出力につながり、時間の経過とともにシステムの保守や改善が困難になります。

### なぜこの設計なのか？
問題を専門化されたagent（それぞれが明確な役割を持つ）に分割することで、以下が得られます：

- **より深く、高品質な調査：** 各agentは自分の領域に集中でき、その仕事に適したツールとプロンプトを使用できます。PMagentはこれらの視点をまとめて、より微妙で堅牢な答えを提供します。

- **モジュール性と明確性：** 他に影響を与えることなく、1つのagentを更新、テスト、または改善できます。これにより、ニーズの進化に応じてシステムの保守と拡張が容易になります。

- **並列処理による高速な結果：** 独立したagentが同時に作業できるため、複雑で多部分にわたる分析を完了する時間が劇的に短縮されます。

- **一貫性と監査可能性：** 構造化されたプロンプト駆動のワークフローにより、すべての実行がベストプラクティスに従い、デバッグが容易で、信頼でき、レビュー可能な出力を生成することが保証されます。

このアプローチは、深さ、専門化、信頼性を求めるあらゆるアプリケーションに理想的です—調査アシスタント、意思決定支援ツール、または専門家のコラボレーションとオーケストレーションから恩恵を受けるシステムを構築する場合でも。

**実践での実装方法：**
- 各スペシャリストagent（Fundamental、Macro、Quantitative）は、SDKの`function_tool`デコレーターを使用してカスタム名と説明を持つ呼び出し可能なツールとしてラップされます。これにより、PMagentのツールセットが明示的でLLMフレンドリーになります。

- Head PMagentは`run_all_specialists_parallel`ツールを使用して3つのスペシャリストすべてを同時に呼び出し、最大速度と効率のために`parallel_tool_calls=True`を活用します。

- agentのプロンプトはmarkdownファイル（`pm_base.md`）から読み込まれ、会社の哲学だけでなく、詳細なツール使用ルールとステップバイステップのワークフローもエンコードします。これにより、すべての実行が一貫性があり、監査可能で、ベストプラクティスに沿ったものになることが保証されます。

- スペシャリストの出力を収集してレビューした後、PMagentは専用のメモエディターツールを使用して投資レポートを組み立て、フォーマットし、完成させます。この関心の分離により、ワークフローがモジュラーで拡張しやすく保たれます。

- システムは拡張性を考慮して設計されています：新しいスペシャリストagentを追加したり、ツールを交換したり、全体のオーケストレーションロジックを壊すことなくプロンプトを更新したりできます。すべてのツール呼び出し、agentの決定、出力は完全な透明性とデバッグのためにOpenAI Tracesでキャプチャされます。

これらの実装選択は上記の利点を直接サポートし、保守、監査、改善が容易な深く、モジュラーで信頼性の高いマルチagent調査ワークフローを可能にします。

### Head Portfolio Manager Agent: コード

```python
from agents import Agent, ModelSettings, function_tool
from utils import load_prompt, DISCLAIMER

def build_head_pm_agent(fundamental, macro, quant, memo_edit_tool):
    def make_agent_tool(agent, name, description):
        @function_tool(name_override=name, description_override=description)
        async def agent_tool(input):
            return await specialist_analysis_func(agent, input)
        return agent_tool
    fundamental_tool = make_agent_tool(fundamental, "fundamental_analysis", "ファンダメンタル分析セクションを生成します。")
    macro_tool = make_agent_tool(macro, "macro_analysis", "マクロ環境セクションを生成します。")
    quant_tool = make_agent_tool(quant, "quantitative_analysis", "定量分析セクションを生成します。")

    @function_tool(name_override="run_all_specialists_parallel", description_override="3つの専門分析（ファンダメンタル、マクロ、定量）を並列実行し、結果を辞書として返します。")
    async def run_all_specialists_tool(fundamental_input, macro_input, quant_input):
        return await run_all_specialists_parallel(
            fundamental, macro, quant,
            fundamental_input, macro_input, quant_input
        )

    return Agent(
        name="Head Portfolio Manager Agent",
        instructions=(load_prompt("pm_base.md") + DISCLAIMER),
        model="gpt-4.1",
        tools=[fundamental_tool, macro_tool, quant_tool, memo_edit_tool, run_all_specialists_tool],
        model_settings=ModelSettings(parallel_tool_calls=True, tool_choice="auto", temperature=0)
    )
    ```

### ヘッドPMシステムプロンプト：ベストプラクティスの徹底

PMエージェントのシステムプロンプト（`prompts/pm_base.md`を参照）は、ワークフローの中核です。これには以下が含まれています：
- 企業の哲学（独創性、リスク認識、コンセンサスへの挑戦）
- 明確なツール使用ルール（並列ツールを使用するタイミング、入力の構造化方法）
- 堅牢な多段階ワークフロー（タスクタイプの決定、ガイダンスの提供、出力のレビュー、メモの組み立て、欠損データの処理）

このプロンプトにより、すべての実行が以下を保証します：
- **一貫性：** 毎回同じ高い基準とプロセスが守られる
- **監査可能性：** 各ステップ、ツール呼び出し、決定がトレースで確認できる
- **高品質：** 出力は独創的で、リスクを認識し、厳密にレビューされている

In [ ]:
# Render the actual system prompt used by the Head Portfolio Manager agent
from pathlib import Path
from IPython.display import Markdown, display

pm_prompt_path = Path("prompts/pm_base.md")
if pm_prompt_path.exists():
    with pm_prompt_path.open("r", encoding="utf-8") as f:
        content = f.read()
    display(Markdown(content))
else:
    print("System prompt not found at prompts/pm_base.md")

---

## 出力例

以下は、ワークフローを通じて生成された投資レポートの例です。あなたの出力は、ディレクトリ内の`outputs`フォルダに書き込まれます。

<details>
<summary>投資メモを展開するにはクリック</summary>

# 投資メモ: Alphabet Inc. (GOOGL) – 計画された金利引き下げの影響 (2025年5月)

## エグゼクティブサマリー

Alphabet Inc. (GOOGL) は現在1株あたり$171.42で取引されており、時価総額は$1.88兆、P/E比は16.91です。投資テーゼは適度に建設的です：連邦準備制度理事会による計画された金利引き下げは軽微な追い風ですが、GOOGLの価格動向の主要な推進力ではありません。当社のビジョンと完全に一致した最も独創的で差別化された洞察は、GOOGLの金利に対する直接的な感応度は控えめ（10年債利回りとの最大週次相関は約0.29）であり、真のリスク/リワードはAI主導の成長の持続可能性、セクターローテーション、規制の逆風にかかっているということです。このテーゼは堅調なテクニカル、強固なファンダメンタルズ、圧倒的にポジティブなアナリストセンチメントに支えられていますが、AI楽観論が薄れるリスクやマクロ/規制ショックの出現によって抑制されています。コンセンサス見解は証拠によって正当化されています：GOOGLの事業は回復力を保っていますが、金利引き下げがテクノロジーを刺激できない、またはセクターローテーションがリターンを制限するというバリアント見解は無視すべきではありません。主要なリスクには規制措置、マクロ経済の不確実性、AI物語の転換の可能性が含まれます。最良のケースでは、GOOGLは2025年末までに$200–$210に到達する可能性があり、最悪のケースでは$160–$170の再テストが妥当です。このメモは、シナリオプランニング、独創的な定量分析、コンセンサスとバリアント見解の批判的評価に焦点を当てることで、当社のビジョンを体現しています。

## ファンダメンタルズの視点

Alphabetの中核事業は、デジタル広告（Google検索、YouTube）における支配的地位と、成長するクラウドおよびAIセグメントによって推進されています。最新四半期（2025年Q1）時点で、売上高は$902億、純利益は$345億、EPSは$2.81、純利益率は38.3%でした。利益率は過去1年間で改善しており、同社のスケールとAIおよびクラウドにおけるリーダーシップは持続可能な堀を提供しています。しかし、最近のアナリスト目標株価は下方修正されており（Bernstein: $165、UBS: $209、Wolfe: $210）、規制およびマクロ経済リスクに対する慎重さを反映しています。コンセンサス見解は正当化されています：Alphabetの財務力とイノベーションは明確ですが、規制の精査とマクロの逆風（例：景気後退時の広告予算削減）は現実のリスクです。最も独創的な洞察は、同社の適応とイノベーション能力であり、一部のリスクを軽減する可能性があります。分析は証拠に基づいており、最近の四半期データは安定または改善する利益率を示しています：

| Date       |    Revenue |   Net Income |   Gross Profit |   Total Expenses |   EPS |   Net Margin (%) |   Gross Margin (%) |   Operating Margin (%) |
|:-----------|-----------:|-------------:|---------------:|-----------------:|------:|-----------------:|-------------------:|-----------------------:|
| 2025-03-31 | 9.0234e+10 |   3.454e+10  |     5.3873e+10 |       5.9628e+10 |  2.81 |          38.28 |            59.70 |                33.92 |
| 2024-12-31 | 9.6469e+10 |   2.6536e+10 |     5.5856e+10 |       6.5497e+10 |  2.15 |          27.51 |            57.90 |                32.11 |
| 2024-09-30 | 8.8268e+10 |   2.6301e+10 |     5.1794e+10 |       5.9747e+10 |  2.12 |          29.80 |            58.68 |                32.31 |
| 2024-06-30 | 8.4742e+10 |   2.3619e+10 |     4.9235e+10 |       5.7317e+10 |  1.89 |          27.87 |            58.10 |                32.36 |
| 2024-03-31 | 8.0539e+10 |   2.3662e+10 |     4.6827e+10 |       5.5067e+10 |  1.89 |          29.38 |            58.14 |                31.63 |

最近のアナリストセンチメントは圧倒的にポジティブで、現在56の買い推奨、12のホールド、0の売り推奨となっています：

| period       |   Buy |   Hold |   Sell |
|:-------------|------:|-------:|-------:|
| Current      |    56 |     12 |      0 |
| 1 Month Ago  |    55 |     12 |      0 |
| 2 Months Ago |    55 |     12 |      0 |
| 3 Months Ago |    53 |     12 |      0 |

ファンダメンタル見解は、証拠、シナリオプランニングに焦点を当て、単純にコンセンサスに従わないことで、当社のビジョンと一致しています。当社ビジョンからの主な乖離は、分析が規制やマクロショックの影響を考慮しなかった場合ですが、これはここで対処されています。

## マクロの視点

マクロ経済環境は複合的です。米国の実質GDPは拡大しており（$23.5兆、2025年Q1）、失業率は低く（4.2%）、インフレは高止まりしています（CPI: 320.3）。連邦準備制度理事会は金利を4.25–4.50%に維持し、忍耐強いスタンスと進化するリスクへの焦点を保っています。米ドルは強く（DXY: 123.4）、最近の関税は不確実性をもたらしています。投資家は米国テクノロジーからアジア株式にローテーションしており、高い評価額と海外でのより良い成長見通しに対する懸念を反映しています。コンセンサスマクロ見解は金利引き下げがテクノロジー評価額を支援するというものですが、当社のビジョンに支えられたバリアント見解は、セクターローテーションと貿易政策がこれらの利益を相殺する可能性があるというものです。テールリスクシナリオには、金利引き下げがGOOGLを支援するベースケース（$180–$190目標）と、貿易緊張やセクターローテーションがリターンを制限するダウンサイドが含まれます。分析は証拠に基づいており、FREDデータと最近の政策声明を使用し、最良および最悪のケースシナリオの両方を明示的に考慮しています。マクロ見解は、コンセンサスに挑戦し、複数の結果を計画することで、当社のビジョンと完全に一致しています。

## 定量的視点

定量分析により、GOOGLの金利に対する直接的な感応度は控えめであることが確認されています。10年債利回りとの平均週次相関は0.29、連邦資金金利との相関は0.05であり、金利変動がGOOGLのリターンの主要な推進力ではないことを示しています。テクニカルは堅調です：GOOGLは主要移動平均を上回り、モメンタムはポジティブ、ボラティリティは適度です。シナリオ分析では、金利引き下げは軽微な追い風ですが、この動きが既に織り込まれているか、テクニカルが破綻した場合、5–10%の調整が可能です。アナリストセンチメントは強くポジティブで、ファンダメンタルズ（売上高、利益率）は改善しています。定量的要約統計：

| Metric                                  |     Value |
|:----------------------------------------|----------:|
| Mean daily corr (FEDFUNDS, GOOGL)       | 0.05 |
| Mean daily reg slope (FEDFUNDS, GOOGL)  | 0.02 |
| Mean daily corr (DGS10, GOOGL)          | 0.13 |
| Mean daily reg slope (DGS10, GOOGL)     | 0.05 |
| Mean weekly corr (FEDFUNDS, GOOGL)      | 0.05 |
| Mean weekly reg slope (FEDFUNDS, GOOGL) | 0.03 |
| Mean weekly corr (DGS10, GOOGL)         | 0.29 |
| Mean weekly reg slope (DGS10, GOOGL)    | 0.09 |

主要なチャートと画像：

![GOOGL Daily Returns](../../../images/multi_agent_collab_googl_daily_returns.png)
![GOOGL Moving Averages](../../../images/multi_agent_collab_googl_moving_averages.png)
![GOOGL RSI](../../../images/multi_agent_collab_googl_rsi.png)
![GOOGL Rolling Volatility](../../../images/multi_agent_collab_googl_rolling_volatility.png)
![Cumulative Return Comparison](../../../images/multi_agent_collab_cumulative_return_comparison.png)
![Rolling Volatility Comparison](../../../images/multi_agent_collab_rolling_volatility_comparison.png)
![Rolling Corr/Reg Daily Fed Funds](../../../images/multi_agent_collab_rolling_corr_reg_daily_fedfunds.png)
![Rolling Corr/Reg Daily 10Y](../../../images/multi_agent_collab_rolling_corr_reg_daily_dgs10.png)
![Rolling Corr/Reg Weekly Fed Funds](../../../images/multi_agent_collab_rolling_corr_reg_weekly_fedfunds.png)
![Rolling Corr/Reg Weekly 10Y](../../../images/multi_agent_collab_rolling_corr_reg_weekly_dgs10.png)
![GOOGL Quarterly Trends](../../../images/multi_agent_collab_GOOGL_quarterly_trends.png)
![GOOGL Quarterly Margins](../../../images/multi_agent_collab_GOOGL_quarterly_margins.png)
![GOOGL Analyst Recommendations Trend](../../../images/multi_agent_collab_GOOGL_analyst_recommendations_trend.png)

定量的見解は、シナリオ分析と控えめな金利感応度への焦点において独創的であり、単純にコンセンサスに従わないことで当社のビジョンと一致しています。制限事項には、パンデミック後の短いデータ期間と、GOOGLの価格が金利を超えた複数の要因（AI、広告市場、規制）によって推進されているという事実が含まれます。

## ポートフォリオマネージャーの視点

PMの統合は、3つの専門セクションすべてが適度に建設的な見通しで収束し、現実的な2025年末目標株価を$190–$210としていることです。最も独創的な洞察は、GOOGLの直接的な金利感応度は控えめであり、真のリスクはAI主導の成長が継続できるか、またはセクターローテーションと規制の逆風がリターンを制限するかということです。定量セクションは堅調なテクニカルとセンチメントを強調することに優れていますが、ダウンサイドシナリオでの$160–$170再テストのリスクも指摘しています。ファンダメンタルとマクロセクションは、規制と貿易政策の監視の重要性を強調しています。大型テクノロジー株でアンダーウェイトの場合、今は合理的なエントリーポイントですが、ポジションサイジングはセクターローテーションやマクロ失望のリスクを反映すべきです。金利引き下げがテクノロジーを刺激できない、またはAI物語の転換というバリアント見解は無視すべきではありません。ポジションサイジングとリスク管理が鍵であり、シナリオプランニングと差別化された洞察という当社のビジョンと完全に一致しています。

## 推奨と質問への回答

推奨は、特に大型テクノロジー株でアンダーウェイトの場合、GOOGLへのエクスポージャーを維持または適度に増加させることで、ベースケースでの2025年末目標株価は$200–$210です。これは、単純にコンセンサスに従うのではなく、独創的で証拠に基づくシナリオ分析に焦点を当てることで、当社のビジョンを体現しています。推奨は堅調なファンダメンタルズ、ポジティブなテクニカル、強いアナリストセンチメントによって正当化されていますが、セクターローテーション、規制措置、またはAI物語の転換のリスクによって抑制されています。これらのリスクが現実化した場合、$160–$170の再テストが可能です。サイジングとリスク管理はこれらのシナリオを反映すべきです。このアプローチは差別化され、証拠主導であり、当社のビジョンと完全に一致しています。

**メモ終了**

*免責事項：私はAI言語モデルであり、登録投資顧問ではありません。提供される情報は教育的で一般的な性質のものです。投資決定を行う前に、資格のある金融専門家にご相談ください。*

</details>

## エージェント構築時のベストプラクティス

最も効果的なエージェントシステムは、モジュラーエージェント設計、明確なツール定義、並列実行、および構造化されたプロンプトを組み合わせています。このアプローチ（OpenAI Agents SDKの中核）により、ワークフローが堅牢で、スケーラブルで、デバッグや拡張が容易になります。

**これらのベストプラクティスを可能にするOpenAI Agents SDKの主要機能：**
- **エージェントループ：** ツール呼び出し、LLM推論、ワークフロー制御を自動的に処理します。
- **Pythonファーストオーケストレーション：** 馴染みのあるPythonパターンを使用してエージェントを連鎖、構成、オーケストレーションします。
- **ハンドオフ：** 専門化とモジュール性のためにエージェント間でタスクを委任します。
- **ガードレール：** 信頼性のために入力/出力を検証し、エラー時に早期に中断します。
- **関数ツール：** 任意のPython関数をツールとして登録し、自動スキーマと検証を提供します。
- **トレーシング：** 完全な透明性のためにワークフローのすべてのステップを可視化、デバッグ、監視します。

効果的なエージェントシステムを構築するには、よく設計されたツール、思慮深いオーケストレーション、慎重なモデル選択の組み合わせが重要です。この例では、強力な分析とツール使用機能を持つGPT-4.1ファミリーのモデルを使用しています（[GPT-4.1 Prompting Guide](https://cookbook.openai.com/examples/gpt4-1_prompting_guide)を参照）。より深いアーキテクチャのベストプラクティスについては、付属の[A Practical Guide to Building Agents (PDF)](https://cdn.openai.com/business-guides-and-resources/a-practical-guide-to-building-agents.pdf)をご覧ください。これらの要素を組み合わせることで、堅牢で、スケーラブルで、デバッグや拡張が容易なシステムを得ることができます。

ぜひご自身の投資に関する質問でサンプルを試してみて、フィードバックをお聞かせください！楽しく構築してください。

---

## 参考資料とベストプラクティス

- [OpenAI Agents SDK Documentation](https://openai.github.io/openai-agents-python/)
- [OpenAI Agents SDK: Multi-Agent Orchestration](https://openai.github.io/openai-agents-python/multi_agent/)
- [OpenAI Agents SDK: Tool List](https://openai.github.io/openai-agents-python/tools/)
- [OpenAI Agents SDK: MCP Documentation](https://openai.github.io/openai-agents-python/mcp/)

- [MCP Spec](https://spec.modelcontextprotocol.io/specification/2024-11-05/architecture/)
- [OpenAI Cookbook](https://github.com/openai/openai-cookbook)
- ([GPT-4.1 Prompting Guide](https://cookbook.openai.com/examples/gpt4-1_prompting_guide))
- [A Practical Guide to Building Agents (PDF)](https://cdn.openai.com/business-guides-and-resources/a-practical-guide-to-building-agents.pdf)

---